# Funciones Iniciales

In [1]:
using HypergeometricFunctions
using LinearAlgebra

In [2]:
function doublefactorial(n)
    if (n <= 1)
        return 1
    end
    return n * doublefactorial(n - 2)
end

doublefactorial (generic function with 1 method)

In [3]:
F_n(n,x) = HypergeometricFunctions.M(n+1/2,n+3/2,-x)/(2*n+1)

F_n (generic function with 1 method)

## Shell

In [4]:
struct shell
    l::Int8
    exp::Array{Float64}
    coef::Array{Float64}
    coord::Array{Float64}
    N::Array{Float64}
    orientaciones::Array{Array{Int}}
end

shell(l, exp, coef, coord) = shell(l, exp, coef, coord, ones(Int((l+1)*(l+2)/2)),[[lx,ly,l-lx-ly] for lx in l:-1:0 for ly in l-lx:-1:0])

function normalized_shell(l,exp,coef,coord)

    #normalizacion
    for i in eachindex(exp)
        coef[i] = coef[i]*(2*exp[i]/pi)^(3/4)*(4*exp[i])^(l/2)
    end
    
    norm = 0
    for j in eachindex(exp)
        for k in eachindex(exp)
            norm += coef[j]*coef[k]*(pi/(exp[j]+exp[k]))^(3/2)/(2*(exp[j]+exp[k]))^(l)
        end
    end

    for i in eachindex(exp)
        coef[i] = coef[i]/sqrt(norm)
    end
        
    orientaciones = []
    for lx in l:-1:0
        for ly in l-lx:-1:0
            lz = l-lx-ly
            append!(orientaciones,[[lx,ly,lz]])      
        end
    end        

    N = ones(Int((l+1)*(l+2)/2))
    for (i,(lx,ly,lz)) in enumerate(orientaciones)
        N[i] = sqrt(1/(doublefactorial(2*lx-1)*doublefactorial(2*ly-1)*doublefactorial(2*lz-1)))
    end
    
    return shell(l, exp, coef, coord, N, orientaciones)
    
end

normalized_shell (generic function with 1 method)

In [5]:
g_a = normalized_shell(0,[0.1,0.2],[1.1,1.2],[1.0,1.0,1.0])
g_b = normalized_shell(0,[0.3,0.4],[1.3,1.4],[2.0,2.0,2.0])

shell(0, [0.3, 0.4], [0.13963711261732617, 0.18659052894127096], [2.0, 2.0, 2.0], [1.0], Array{Int64}[[0, 0, 0]])

In [7]:
g_a = shell(1,[0.5,0.7],[1.0,1.0],[1.0,1.5,2.0])

shell(1, [0.5, 0.7], [1.0, 1.0], [1.0, 1.5, 2.0], [1.0, 1.0, 1.0], Array{Int64}[[1, 0, 0], [0, 1, 0], [0, 0, 1]])

## Shell Pair

In [7]:
struct shell_pair
    
    g_a::shell
    g_b::shell
    
    exp::Array{Float64}
    coef::Array{Float64}
    coord::Array{Array{Float64}}
    alpha::Array{Float64}
    
end
    
function build_shell_pair(g_a, g_b)
    exp = vec((g_a.exp .+ g_b.exp')')
    coef = vec((g_a.coef .* g_b.coef')')

    coord = []
    for i in eachindex(g_a.exp)
        for j in eachindex(g_b.exp)
            append!(coord,[(g_a.exp[i]*g_a.coord+g_b.exp[j]*g_b.coord)/(g_a.exp[i]+g_b.exp[j])])
        end
    end
            
    alpha = vec(((g_a.exp .* g_b.exp')./(g_a.exp .+ g_b.exp'))')
    
    return shell_pair(g_a, g_b, exp, coef, coord, alpha)

end

build_shell_pair (generic function with 1 method)

In [8]:
pairAB = build_shell_pair(g_a,g_b)

shell_pair(shell(0, [0.5, 0.7], [1.0, 1.0], [1.0, 1.5, 2.0], [1.0], Array{Int64}[[0]]), shell(0, [0.3, 0.4], [0.13963711261732617, 0.18659052894127096], [2.0, 2.0, 2.0], [1.0], Array{Int64}[[0, 0, 0]]), [0.8, 0.9, 1.0, 1.1], [0.13963711261732617, 0.18659052894127096, 0.13963711261732617, 0.18659052894127096], Array{Float64}[[1.375, 1.6875, 2.0], [1.4444444444444444, 1.7222222222222223, 2.0], [1.2999999999999998, 1.65, 2.0], [1.3636363636363635, 1.6818181818181817, 2.0]], [0.18749999999999997, 0.22222222222222224, 0.21, 0.2545454545454545])

## Creador de Shell

In [9]:
function get_set_of_shells(element,coord,basis_name)

    element = uppercase(element)
    shells = []
    
    # Basis file
    f = open("basis/"*basis_name*".bas","r")
    lines = readlines(f)
    close(f)
    
    # Find_atom
    atom_line = 0
    for (i,line) in enumerate(lines)
        splitted_line = split(line)
        if(size(splitted_line)[1]>0)
            if(occursin("O-",splitted_line[1]))
                if(splitted_line[2] == element)
                    atom_line = i
                    break
                end
            end
        end
    end

    # Pass the scheme contraction info line
    num_of_shells = parse(Int,lines[atom_line + 2])
    
    # Read shells
    new_shell = true
    idx_shell = 0
    idx_prim = 0
    exp = []
    coef = []
    idx,l,num_of_primitives = 0,0,0
    for line in lines[atom_line + 3:size(lines)[1]]
        splitted_line = split(line)
        #If new shell, read angular momentum and number of primitives
        if new_shell
            idx_shell += 1
            idx_prim = 0        
            idx,l,num_of_primitives = parse(Int,splitted_line[1]),parse(Int,splitted_line[2]),parse(Int,splitted_line[3])
            exp = []
            coef = []
            new_shell = false
            continue
        end
        #If not a new shell, read exponents and coefficients
        if !new_shell
            idx_prim += 1
            append!(exp,parse(Float64,splitted_line[1]))           
            append!(coef,parse(Float64,splitted_line[2]))
            if(idx_prim == num_of_primitives)
                push!(shells,normalized_shell(l,exp,coef,coord))
                #If all shells readed, break, else, read new shell
                if(idx_shell == num_of_shells)
                    break
                else
                    new_shell = true
                end
            end
            continue
        end
    end
            
    return shells
                        
end

get_set_of_shells (generic function with 1 method)

In [10]:
shells = get_set_of_shells("C",[0.0,0.0,0.0],"aug-cc-pVDZ")

9-element Vector{Any}:
 shell(0, [6665.0, 1000.0, 228.0, 64.71, 21.06, 7.495, 2.797, 0.5215, 0.1596], [0.3638031159963351, 0.6753920199851438, 1.1323009845638856, 1.654004043786724, 1.9249784580083262, 1.4481493327119654, 0.43942743105618515, 0.006649803316796763, -0.0005742641091064987], [0.0, 0.0, 0.0], [1.0], Array{Int64}[[0, 0, 0]])
 shell(0, [6665.0, 1000.0, 228.0, 64.71, 21.06, 7.495, 2.797, 0.5215, 0.1596], [-0.07675616263267492, -0.14625680841141203, -0.2394070358248575, -0.3790690855337743, -0.44810373279382504, -0.48420052512242984, -0.1961680961614069, 0.23816176040401543, 0.10446821641194678], [0.0, 0.0, 0.0], [1.0], Array{Int64}[[0, 0, 0]])
 shell(0, [0.1596], [0.1799636353308006], [0.0, 0.0, 0.0], [1.0], Array{Int64}[[0, 0, 0]])
 shell(0, [0.0469], [0.07182733022052408], [0.0, 0.0, 0.0], [1.0], Array{Int64}[[0, 0, 0]])
 shell(1, [9.439, 2.002, 0.5456, 0.1517], [0.8987219849137384, 0.7110710327814355, 0.3399170263839058, 0.06327014302166259], [0.0, 0.0, 0.0], [1.0, 1.0, 1.

In [11]:
function get_all_shells_from_xyz(molecule,basis_name)
    molecule = split(molecule,"\n")    
    natoms = size(molecule)[1]
    shells = []
    for iatom in 1:natoms
        atom = split(molecule[iatom])
        if size(atom)[1] >= 1
            shell = get_set_of_shells(atom[1],[parse(Float64,atom[2]),parse(Float64,atom[3]),parse(Float64,atom[4])],basis_name)
            push!(shells,shell)
        end
    end
    return shells        
end

get_all_shells_from_xyz (generic function with 1 method)

In [12]:
molecule = """
O   0.000000  0.000000  0.000000
H   0.277400  0.892900  0.254400
H   0.606800 -0.238300 -0.716900
"""

"O   0.000000  0.000000  0.000000\nH   0.277400  0.892900  0.254400\nH   0.606800 -0.238300 -0.716900\n"

In [13]:
shells = get_all_shells_from_xyz(molecule,"aug-cc-pVDZ")

3-element Vector{Any}:
 Any[shell(0, [11720.0, 1759.0, 400.8, 113.7, 37.03, 13.27, 5.025, 1.013, 0.3023], [0.569986149793314, 1.058880501673631, 1.7771678966453877, 2.600709693959018, 3.028357204192797, 2.223503905508639, 0.6481188547796517, 0.011124247231272932, -0.0007511028383196536], [0.0, 0.0, 0.0], [1.0], Array{Int64}[[0, 0, 0]]), shell(0, [11720.0, 1759.0, 400.8, 113.7, 37.03, 13.27, 5.025, 1.013, 0.3023], [-0.12844755012231768, -0.24449099282587303, -0.40009729327831506, -0.6381663487030188, -0.7587848290723769, -0.8196486224662376, -0.27975700946278415, 0.4011060836621272, 0.1664219730305194], [0.0, 0.0, 0.0], [1.0], Array{Int64}[[0, 0, 0]]), shell(0, [0.3023], [0.2905619240088511], [0.0, 0.0, 0.0], [1.0], Array{Int64}[[0, 0, 0]]), shell(0, [0.07896], [0.10616110906976056], [0.0, 0.0, 0.0], [1.0], Array{Int64}[[0, 0, 0]]), shell(1, [17.7, 3.854, 1.046, 0.2753], [2.226162227432104, 1.7619764776359819, 0.7670792917006999, 0.13090512359254836], [0.0, 0.0, 0.0], [1.0, 1.0, 1.0], A

# MD

In [14]:
g_a = normalized_shell(0,[13.01, 1.962, 0.4446, 0.122],[0.01968500,0.1379770,0.4781480,0.5012400],[0.0,0.0,0.0])
g_b = normalized_shell(0,[0.07896],[1.0],[0,0,0])
g_c = normalized_shell(0,[0.141],[1.0],[0.6068,-0.2383,-0.7169])
g_d = normalized_shell(0,[0.07896],[1.0],[0,0,0])

shell(0, [0.07896], [0.10616110906976056], [0.0, 0.0, 0.0], [1.0], Array{Int64}[[0, 0, 0]])

In [15]:
function compute_E(t,la,lb,pair,i,xyz)
    
    if(t<0)
        return 0
    elseif(t>la+lb)
        return 0
    elseif(la>0)
        E_im1j_tm1 = compute_E(t-1,la-1,lb,pair,i,xyz)
        E_im1j_t = compute_E(t,la-1,lb,pair,i,xyz)
        E_im1j_tp1 = compute_E(t+1,la-1,lb,pair,i,xyz)
        return 1/(2*pair.exp[i])*E_im1j_tm1 + (pair.coord[i][xyz]-pair.g_a.coord[xyz])*E_im1j_t + (t+1)*E_im1j_tp1
    elseif(lb>0)
        E_ijm1_tm1 = compute_E(t-1,la,lb-1,pair,i,xyz)
        E_ijm1_t = compute_E(t,la,lb-1,pair,i,xyz)
        E_ijm1_tp1 = compute_E(t+1,la,lb-1,pair,i,xyz)    
        return 1/(2*pair.exp[i])*E_ijm1_tm1 + (pair.coord[i][xyz]-pair.g_b.coord[xyz])*E_ijm1_t + (t+1)*E_ijm1_tp1
    elseif(t==0 && la==0 && lb==0)
        return exp(-pair.alpha[i]*(pair.g_b.coord[xyz]-pair.g_a.coord[xyz])^2)
    end
end

compute_E (generic function with 1 method)

In [16]:
function compute_R(v_x,v_y,v_z,n,pairAB,pairCD,i,j,alpha,Rpq)

    if(v_x<0 || v_y<0 || v_z<0)
        return 0
    elseif(v_x!=0)
        R_np1_vm2 = compute_R(v_x-2,v_y,v_z,n+1,pairAB,pairCD,i,j,alpha,Rpq)
        R_np1_vm1 = compute_R(v_x-1,v_y,v_z,n+1,pairAB,pairCD,i,j,alpha,Rpq)
        return (v_x-1)*R_np1_vm2 + Rpq[1]*R_np1_vm1
    elseif(v_y!=0)
        R_np1_vm2 = compute_R(v_x,v_y-2,v_z,n+1,pairAB,pairCD,i,j,alpha,Rpq)
        R_np1_vm1 = compute_R(v_x,v_y-1,v_z,n+1,pairAB,pairCD,i,j,alpha,Rpq)
        return (v_y-1)*R_np1_vm2 + Rpq[2]*R_np1_vm1
    elseif(v_z!=0)
        R_np1_vm2 = compute_R(v_x,v_y,v_z-2,n+1,pairAB,pairCD,i,j,alpha,Rpq)
        R_np1_vm1 = compute_R(v_x,v_y,v_z-1,n+1,pairAB,pairCD,i,j,alpha,Rpq)
        return (v_z-1)*R_np1_vm2 + Rpq[3]*R_np1_vm1
    else
        return (-2*alpha)^n*2*pi^(5/2)/(pairAB.exp[i]*pairCD.exp[j]*sqrt(pairAB.exp[i]+pairCD.exp[j]))*F_n(n,alpha*norm(Rpq)^2)
    end
end

compute_R (generic function with 1 method)

In [17]:
function MD(g_a,g_b,g_c,g_d)

    pairAB = build_shell_pair(g_a,g_b)
    pairCD = build_shell_pair(g_c,g_d)

    result = zeros(size(g_a.orientaciones)[1],size(g_b.orientaciones)[1],size(g_c.orientaciones)[1],size(g_d.orientaciones)[1])
    for (a,(lax,lay,laz)) in enumerate(g_a.orientaciones)
        for (b,(lbx,lby,lbz)) in enumerate(g_b.orientaciones)
            for (c,(lcx,lcy,lcz)) in enumerate(g_c.orientaciones)
                for (d,(ldx,ldy,ldz)) in enumerate(g_d.orientaciones)
                    integral = 0.0
                    for i in eachindex(pairAB.exp)
                        aux7 = 0
                        for t_z in 0:laz+lbz
                            Ez_ij_t = compute_E(t_z,laz,lbz,pairAB,i,3)
                            aux6 = 0    
                            for t_y in 0:lay+lby
                                Ey_ij_t = compute_E(t_y,lay,lby,pairAB,i,2)
                                aux5 = 0        
                                for t_x in 0:lax+lbx
                                    Ex_ij_t = compute_E(t_x,lax,lbx,pairAB,i,1)
                                    aux4 = 0
                                    for j in eachindex(pairCD.exp)    
                                        alpha = pairAB.exp[i]*pairCD.exp[j]/(pairAB.exp[i]+pairCD.exp[j])
                                        Rpq = pairAB.coord[i]-pairCD.coord[j]
                                        aux3 = 0
                                        for u_z in 0:lcz+ldz
                                            Ez_kl_u = compute_E(u_z,lcz,ldz,pairCD,j,3)
                                            aux2 = 0
                                            for u_y in 0:lcy+ldy
                                                Ey_kl_u = compute_E(u_y,lcy,ldy,pairCD,j,2)
                                                aux1 = 0
                                                for u_x in 0:lcx+ldx
                                                    Ex_kl_u = compute_E(u_x,lcx,ldx,pairCD,j,1)
                                                    aux1 += (-1)^(u_x+u_y+u_z)*Ex_kl_u*compute_R(t_x+u_x,t_y+u_y,t_z+u_z,0,pairAB,pairCD,i,j,alpha,Rpq)
                                                end
                                                aux2 += Ey_kl_u*aux1
                                            end
                                            aux3 += Ez_kl_u*aux2
                                        end
                                        aux4 += aux3*pairCD.coef[j]
                                    end
                                    aux5 += Ex_ij_t*aux4
                                end
                                aux6 += Ey_ij_t*aux5
                            end
                            aux7 += Ez_ij_t*aux6
                        end
                        integral += aux7*pairAB.coef[i]
                    end
                    result[a,b,c,d] = g_a.N[a]*g_b.N[b]*g_c.N[c]*g_d.N[d]*integral
                end
            end
        end
    end

    return result
    
end

MD (generic function with 1 method)

In [18]:
using BenchmarkTools

@btime MD(g_a,g_b,g_c,g_d)

  4.329 μs (512 allocations: 17.84 KiB)


1×1×1×1 Array{Float64, 4}:
[:, :, 1, 1] =
 0.28447427867464337

In [19]:
function build_R(lt,alpha,Rpq,F,pairAB,pairCD)
    R = [[[[[[0.0 for lz in 0:lt-n-lx-ly] for ly in 0:lt-n-lx] for lx in 0:lt-n] for n in 0:lt] for jj in eachindex(pairCD.exp)] for ii in eachindex(pairAB.exp)]
    for ii in eachindex(pairAB.exp)
        for jj in eachindex(pairCD.exp)
            for nini in 0:lt
                R[ii][jj][nini+1][1][1][1] = (-2*alpha[ii][jj])^nini*2*pi^(5/2)/(pairAB.exp[ii]*pairCD.exp[jj]*sqrt(pairAB.exp[ii]+pairCD.exp[jj]))*F[ii][jj][nini+1]
                l = 0
                for n in nini-1:-1:0
                    l += 1
                    for lx in 0:l
                        for ly in 0:l-lx
                            lz = l-lx-ly
                            if(lz>=2)
                                R[ii][jj][n+1][lx+1][ly+1][lz+1] += (lz-1)*R[ii][jj][n+2][lx+1][ly+1][lz-1]
                            end
                            if(lz>=1)
                                R[ii][jj][n+1][lx+1][ly+1][lz+1] += Rpq[ii][jj][3]*R[ii][jj][n+2][lx+1][ly+1][lz]
                            end
                        end
                        ly = l-lx
                        if(ly>=2)
                            R[ii][jj][n+1][lx+1][l-lx+1][1] += (ly-1)*R[ii][jj][n+2][lx+1][l-lx-1][1]
                        end
                        if(ly>=1)
                            R[ii][jj][n+1][lx+1][l-lx+1][1] += Rpq[ii][jj][2]*R[ii][jj][n+2][lx+1][l-lx][1]
                        end
                    end
                    lx = l
                    if(lx>=2)
                        R[ii][jj][n+1][l+1][1][1] += (lx-1)*R[ii][jj][n+2][l-1][1][1]
                    end
                    if(lx>=1)
                        R[ii][jj][n+1][l+1][1][1] += Rpq[ii][jj][1]*R[ii][jj][n+2][l][1][1]                
                    end
                end
            end
        end
    end
             
    return R

end

build_R (generic function with 1 method)

In [20]:
function build_E(l1,l2,pair)
    E = [[[[[0.0 for t in 0:la+lb] for lb in 0:l2] for la in 0:l1] for xyz in 1:3] for ii in eachindex(pair.exp)]

    for ii in eachindex(pair.exp)
        for xyz in 1:3
            E[ii][xyz][1][1][1] = exp(-pair.alpha[ii]*(pair.g_b.coord[xyz]-pair.g_a.coord[xyz])^2)
            for la in 0:l1
                for lb in 0:l2
                    for t in 0:la+lb
                        if(lb>0)
                            if(t>0)               
                                E[ii][xyz][la+1][lb+1][t+1] += 1/(2*pair.exp[ii])*E[ii][xyz][la+1][lb][t]
                            end
                            if(t<=la+lb-1)
                                E[ii][xyz][la+1][lb+1][t+1] += (pair.coord[ii][xyz]-pair.g_b.coord[xyz])*E[ii][xyz][la+1][lb][t+1]
                            end
                            if(t+1<=la+lb-1)
                                E[ii][xyz][la+1][lb+1][t+1] += (t+1)*E[ii][xyz][la+1][lb][t+2]
                            end
                        elseif(la>0)
                            if(t>0)               
                                E[ii][xyz][la+1][lb+1][t+1] += 1/(2*pair.exp[ii])*E[ii][xyz][la][lb+1][t]
                            end
                            if(t<=la-1+lb)
                                E[ii][xyz][la+1][lb+1][t+1] += (pair.coord[ii][xyz]-pair.g_a.coord[xyz])*E[ii][xyz][la][lb+1][t+1]
                            end
                            if(t+1<=la-1+lb)
                                E[ii][xyz][la+1][lb+1][t+1] += (t+1)*E[ii][xyz][la][lb+1][t+2]
                            end
                        end
                    end
                end
            end
        end
    end
                                
    return E
    
end

build_E (generic function with 1 method)

In [21]:
pairAB = build_shell_pair(g_a,g_b)
pairCD = build_shell_pair(g_c,g_d)

lt = pairAB.g_a.l + pairAB.g_b.l + pairCD.g_a.l + pairCD.g_b.l

# Build boys
alpha = [[pairAB.exp[ii]*pairCD.exp[jj]/(pairAB.exp[ii]+pairCD.exp[jj]) for jj in eachindex(pairCD.exp)] for ii in eachindex(pairAB.exp)]
Rpq = [[pairAB.coord[ii]-pairCD.coord[jj] for jj in eachindex(pairCD.exp)] for ii in eachindex(pairAB.exp)]
F = [[[F_n(n,alpha[ii][jj]*norm(Rpq[ii][jj])^2) for n in 0:lt] for jj in eachindex(pairCD.exp)] for ii in eachindex(pairAB.exp)]

# Build R
R = build_R(lt,alpha,Rpq,F,pairAB,pairCD)

# Build E
Eab = build_E(pairAB.g_a.l,pairAB.g_b.l,pairAB)
Ecd = build_E(pairCD.g_a.l,pairCD.g_b.l,pairCD)

result = zeros((size(g_a.orientaciones)[1],size(g_b.orientaciones)[1],size(g_c.orientaciones)[1],size(g_d.orientaciones)[1]))
for (a,(lax,lay,laz)) in enumerate(g_a.orientaciones)
    for (b,(lbx,lby,lbz)) in enumerate(g_b.orientaciones)
        for (c,(lcx,lcy,lcz)) in enumerate(g_c.orientaciones)
            for (d,(ldx,ldy,ldz)) in enumerate(g_d.orientaciones)

                integral = 0.0
                for ii in eachindex(pairAB.exp)
                    aux7 = 0
                    for t_z in 0:laz+lbz
                        aux6 = 0    
                        for t_y in 0:lay+lby
                            aux5 = 0        
                            for t_x in 0:lax+lbx
                                aux4 = 0
                                for jj in eachindex(pairCD.exp)
                                    aux3 = 0
                                    for u_z in 0:lcz+ldz
                                        aux2 = 0
                                        for u_y in 0:lcy+ldy
                                            aux1 = 0
                                            for u_x in 0:lcx+ldx
                                                aux1 += (-1)^(u_x+u_y+u_z)*Ecd[jj][1][lcx+1][ldx+1][u_x+1]*R[ii][jj][1][t_x+u_x+1][t_y+u_y+1][t_z+u_z+1]
                                            end
                                            aux2 += Ecd[jj][2][lcy+1][ldy+1][u_y+1]*aux1
                                        end
                                        aux3 += Ecd[jj][3][lcz+1][ldz+1][u_z+1]*aux2
                                    end
                                    aux4 += aux3*pairCD.coef[jj]
                                end
                                aux5 += Eab[ii][1][lax+1][lbx+1][t_x+1]*aux4
                            end
                            aux6 += Eab[ii][2][lay+1][lby+1][t_y+1]*aux5
                        end
                        aux7 += Eab[ii][3][laz+1][lbz+1][t_z+1]*aux6
                    end
                    integral += aux7*pairAB.coef[ii]
                end           
                result[a,b,c,d] = g_a.N[a]*g_b.N[b]*g_c.N[c]*g_d.N[d]*integral
                println(integral)      
            end
        end
    end
end
            

0.28447427867464337


## Overlap

In [120]:
g_a = normalized_shell(0,[13.01],[0.01968500],[0.0,0.0,0.0])
g_b = normalized_shell(0,[0.07896],[1.0],[0,0,0])
pairAB = build_shell_pair(g_a,g_b)

shell_pair(shell(0, [13.01], [4.882228084827355], [0.0, 0.0, 0.0], [1.0], Array{Int64}[[0, 0, 0]]), shell(0, [0.07896], [0.10616110906976056], [0.0, 0.0, 0.0], [1.0], Array{Int64}[[0, 0, 0]]), [13.08896], [0.5183027482168051], Array{Float64}[[0.0, 0.0, 0.0]], [0.07848366867955897])

In [121]:
Eab = build_E(pairAB.g_a.l,pairAB.g_b.l,pairAB)

result = zeros((size(g_a.orientaciones)[1],size(g_b.orientaciones)[1]))

for (a,(lax,lay,laz)) in enumerate(g_a.orientaciones)
    for (b,(lbx,lby,lbz)) in enumerate(g_b.orientaciones)
        
        integral = 0.0
        for ii in eachindex(pairAB.exp)
            t_z = 0
            t_y = 0
            t_x = 0
            integral += Eab[ii][1][lax+1][lbx+1][t_x+1]*
                        Eab[ii][2][lay+1][lby+1][t_y+1]*
                        Eab[ii][3][laz+1][lbz+1][t_z+1]*
                        (pi/pairAB.exp[ii])^1.5*pairAB.coef[ii]
        end           
        result[a,b] = g_a.N[a]*g_b.N[b]*integral
        println(result[a,b])      
    end
end

0.06094675816728935


In [24]:
pairAB

shell_pair(shell(0, [13.01, 1.962, 0.4446, 0.122], [0.09610661098332268, 0.16301997479104427, 0.18554502268336198, 0.07374379198350509], [0.0, 0.0, 0.0], [1.0], Array{Int64}[[0, 0, 0]]), shell(0, [0.07896], [0.10616110906976056], [0.0, 0.0, 0.0], [1.0], Array{Int64}[[0, 0, 0]]), [13.08896, 2.04096, 0.52356, 0.20096], [0.010202784410925567, 0.017306381324341667, 0.01969766539043959, 0.007828722743978618], Array{Float64}[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]], [0.07848366867955897, 0.0759052210724365, 0.06705175338070135, 0.04793550955414013])

# Potential

In [25]:
coord = [0,0,0]

3-element Vector{Int64}:
 0
 0
 0

In [26]:
p = pairAB.exp
Rpc = [pairAB.coord[ii]-coord for ii in eachindex(pairAB.exp)]
F = [[F_n(n,p[ii]*norm(Rpc[ii])^2) for n in 0:pairAB.g_a.l + pairAB.g_b.l] for ii in eachindex(pairAB.exp)]

4-element Vector{Vector{Float64}}:
 [1.0]
 [1.0]
 [1.0]
 [1.0]

In [27]:
R = [[[[[0.0 for lz in 0:lt-n-lx-ly] for ly in 0:lt-n-lx] for lx in 0:lt-n] for n in 0:lt] for ii in eachindex(pairAB.exp)]

4-element Vector{Vector{Vector{Vector{Vector{Float64}}}}}:
 [[[[0.0]]]]
 [[[[0.0]]]]
 [[[[0.0]]]]
 [[[[0.0]]]]

In [28]:
function build_R_one(coord,pairAB)
    p = pairAB.exp
    Rpc = [pairAB.coord[ii]-coord for ii in eachindex(pairAB.exp)]
    F = [[F_n(n,p[ii]*norm(Rpc[ii])^2) for n in 0:pairAB.g_a.l + pairAB.g_b.l] for ii in eachindex(pairAB.exp)]
    lt = pairAB.g_a.l + pairAB.g_b.l
    R = [[[[[0.0 for lz in 0:lt-n-lx-ly] for ly in 0:lt-n-lx] for lx in 0:lt-n] for n in 0:lt] for ii in eachindex(pairAB.exp)]
    for ii in eachindex(pairAB.exp)
#        for jj in eachindex(pairCD.exp)
            for nini in 0:lt
                R[ii][nini+1][1][1][1] = (-2*p[ii])^nini*2*pi/p[ii]*F[ii][nini+1]
                l = 0
                for n in nini-1:-1:0
                    l += 1
                    for lx in 0:l
                        for ly in 0:l-lx
                            lz = l-lx-ly
                            if(lz>=2)
                                R[ii][n+1][lx+1][ly+1][lz+1] += (lz-1)*R[ii][n+2][lx+1][ly+1][lz-1]
                            end
                            if(lz>=1)
                                R[ii][n+1][lx+1][ly+1][lz+1] += Rpc[ii][3]*R[ii][n+2][lx+1][ly+1][lz]
                            end
                        end
                        ly = l-lx
                        if(ly>=2)
                            R[ii][n+1][lx+1][l-lx+1][1] += (ly-1)*R[ii][n+2][lx+1][l-lx-1][1]
                        end
                        if(ly>=1)
                            R[ii][n+1][lx+1][l-lx+1][1] += Rpc[ii][2]*R[ii][n+2][lx+1][l-lx][1]
                        end
                    end
                    lx = l
                    if(lx>=2)
                        R[ii][n+1][l+1][1][1] += (lx-1)*R[ii][n+2][l-1][1][1]
                    end
                    if(lx>=1)
                        R[ii][n+1][l+1][1][1] += Rpc[ii][1]*R[ii][n+2][l][1][1]                
                    end
                end
            end
        #end
    end
             
    return R

end

build_R_one (generic function with 1 method)

In [29]:
R = build_R_one(coord,pairAB)
Eab = build_E(pairAB.g_a.l,pairAB.g_b.l,pairAB)

result = zeros((size(g_a.orientaciones)[1],size(g_b.orientaciones)[1]))

for (a,(lax,lay,laz)) in enumerate(g_a.orientaciones)
    for (b,(lbx,lby,lbz)) in enumerate(g_b.orientaciones)
        
        integral = 0.0
        for ii in eachindex(pairAB.exp)
            aux3 = 0
            for t_z in 0:laz+lbz
                aux2 = 0    
                for t_y in 0:lay+lby
                    aux1 = 0       
                    for t_x in 0:lax+lbx
                        aux1 += Eab[ii][1][lax+1][lbx+1][t_x+1]*R[ii][1][t_x+1][t_y+1][t_z+1]
                    end
                    aux2 += Eab[ii][2][lay+1][lby+1][t_y+1]*aux1
                end
                aux3 += Eab[ii][3][laz+1][lbz+1][t_z+1]*aux2
            end
                    
            integral += aux3*pairAB.coef[ii]
        end           
        result[a,b] = g_a.N[a]*g_b.N[b]*integral
        println(result[a,b])      
    end
end

0.5393373370926999


# Cinética

In [203]:
g_a = normalized_shell(1,[13.01],[0.01968500],[0.0,0.0,0.0])
g_b = normalized_shell(1,[0.07896],[1.0],[0,0,0])
pairAB = build_shell_pair(g_a,g_b)

shell_pair(shell(1, [13.01], [35.21978565808279], [0.0, 0.0, 0.0], [1.0, 1.0, 1.0], Array{Int64}[[1, 0, 0], [0, 1, 0], [0, 0, 1]]), shell(1, [0.07896], [0.059662165498468096], [0.0, 0.0, 0.0], [1.0, 1.0, 1.0], Array{Int64}[[1, 0, 0], [0, 1, 0], [0, 0, 1]]), [13.08896], [2.101288680753109], Array{Float64}[[0.0, 0.0, 0.0]], [0.07848366867955897])

In [237]:
function Tij(la,lb,xyz,pairAB)
    
    Eab = build_E(pairAB.g_a.l,pairAB.g_b.l,pairAB)
    
    S = [[[0.0 for j in 0:lb] for i in 0:la] for ii in eachindex(pairAB.exp)]
    T = [[[0.0 for j in 0:lb] for i in 0:la] for ii in eachindex(pairAB.exp)]
    
    for ii in eachindex(pairAB.exp)
        i = 0
        t = 0
        for j in 0:lb
            S[ii][i+1][j+1] = Eab[ii][xyz][i+1][j+1][t+1]*(pi/pairAB.exp[ii])^0.5            
        end
        for i in 1:la
            for j in max(0,lb-la+i):lb
                S[ii][i+1][j+1] = Eab[ii][xyz][i+1][j+1][t+1]*(pi/pairAB.exp[ii])^0.5
            end
        end
    end

    ii = 0
    for jprim in 1:size(pairAB.g_b.exp)[1], iprim in 1:size(pairAB.g_a.exp)[1]
        ii += 1
        i=0
        j=0
        T[ii][i+1][j+1] = (pairAB.g_a.exp[iprim] - 2*pairAB.g_a.exp[iprim]^2 * ((pairAB.coord[ii][xyz]-pairAB.g_a.coord[xyz])^2 + 1/(2*pairAB.exp[ii]))) * S[ii][i+1][j+1]
        for j in 1:lb
            T[ii][i+1][j+1] = (pairAB.coord[ii][xyz]-pairAB.g_b.coord[xyz])*T[ii][i+1][j+1] + pairAB.g_a.exp[iprim]/pairAB.exp[ii]*2*pairAB.g_b.exp[jprim]*S[ii][i+1][j+1] 
            if(j>1)
                T[ii][i+1][j+1] += (j-1)/(2*pairAB.exp[ii])*T[ii][i+1][j-1] - pairAB.g_a.exp[iprim]/pairAB.exp[ii]*(j)*S[ii][i+1][j-1]
            end
        end
        for i in 1:la
            for j in max(0,lb-la+i):lb
                T[ii][i+1][j+1] = (pairAB.coord[ii][xyz]-pairAB.g_a.coord[xyz])*T[ii][i][j+1] + pairAB.g_b.exp[jprim]/pairAB.exp[ii]*2*pairAB.g_a.exp[iprim]*S[ii][i+1][j+1] 
                if(i>1)
                    T[ii][i+1][j+1] += (i-1)/(2*pairAB.exp[ii])*T[ii][i-1][j+1] - pairAB.g_b.exp[jprim]/pairAB.exp[ii]*(i-1)*S[ii][i-1][j+1]
                end
                if(j>0)
                    T[ii][i+1][j+1] += (j)/(2*pairAB.exp[ii])*T[ii][i][j]# - pairAB.g_b.exp[jprim]/pairAB.exp[ii]*(i-1)*S[ii][i-1][j+1]
                end
            end
        end
    end       
        
    return [T[ii][la+1][lb+1] for ii in eachindex(pairAB.exp)]
        
end

Tij (generic function with 1 method)

In [238]:
Eab = build_E(pairAB.g_a.l,pairAB.g_b.l,pairAB)

result = zeros((size(g_a.orientaciones)[1],size(g_b.orientaciones)[1]))

for (a,(lax,lay,laz)) in enumerate(g_a.orientaciones)
    for (b,(lbx,lby,lbz)) in enumerate(g_b.orientaciones)
        
        Tx = Tij(lax,lbx,1,pairAB)
        Ty = Tij(lay,lby,2,pairAB)
        Tz = Tij(laz,lbz,3,pairAB)
        
        integral = 0.0
        for ii in eachindex(pairAB.exp)
            tx,ty,tz = 0,0,0
            Sx = Eab[ii][1][lax+1][lbx+1][tx+1]*(pi/pairAB.exp[ii])^0.5
            Sy = Eab[ii][2][lay+1][lby+1][ty+1]*(pi/pairAB.exp[ii])^0.5
            Sz = Eab[ii][3][laz+1][lbz+1][tz+1]*(pi/pairAB.exp[ii])^0.5
            integral += pairAB.coef[ii]*(Tx[ii]*Sy*Sz+Ty[ii]*Sx*Sz+Tz[ii]*Sx*Sy)
        end           
        result[a,b] = g_a.N[a]*g_b.N[b]*integral
        println(result[a,b])      
    end
end

0.003703965929731752
0.0
0.0
0.0
0.003703965929731752
0.0
0.0
0.0
0.003703965929731752
